In [6]:
import pandas as pd
import numpy as np
import folium
from folium import plugins

In [8]:
df = pd.read_csv('https://brasil.io/dataset/covid19/caso?format=csv')

In [9]:
df.head()

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-04-22,AC,Acrelândia,city,12,0,True,15256.0,1200013.0,78.65758,NaN
1,2020-04-22,AC,Bujari,city,1,0,True,10266.0,1200138.0,9.74089,NaN
2,2020-04-22,AC,Cruzeiro do Sul,city,5,0,True,88376.0,1200203.0,5.65764,NaN
3,2020-04-22,AC,Plácido de Castro,city,21,1,True,19761.0,1200385.0,106.26993,0.0476
4,2020-04-22,AC,Porto Acre,city,1,0,True,18504.0,1200807.0,5.40424,NaN


# IBGE LATITUDE E LONGITUDE

In [10]:
cidades = pd.read_csv('https://raw.githubusercontent.com/sandeco/CanalSandeco/master/covid-19/cidades_brasil.csv')


In [19]:
cidades = cidades.set_index('codigo_ibge')

In [20]:
cidades.head()

,nome,latitude,longitude,capital,codigo_uf
codigo_ibge,,,,,
5200050,Abadia de Goiás,-16.75730,-49.4412,0,52
3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31
5200100,Abadiânia,-16.19700,-48.7057,0,52
3100203,Abaeté,-19.15510,-45.4444,0,31
1500107,Abaetetuba,-1.72183,-48.8788,0,15


# Data preprocessing

In [12]:
cities = df.loc[df.place_type == 'city', :]
cities.place_type.unique()

array(['city'], dtype=object)

In [13]:
len(cities)

24886

In [21]:
cities = cities.join(cidades,on='city_ibge_code')

In [22]:
cities.head()

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,nome,latitude,longitude,capital,codigo_uf
0,2020-04-22,AC,Acrelândia,city,12,0,True,15256.0,1200013.0,78.65758,NaN,Acrelândia,-9.82581,-66.8972,0.0,12.0
1,2020-04-22,AC,Bujari,city,1,0,True,10266.0,1200138.0,9.74089,NaN,Bujari,-9.81528,-67.9550,0.0,12.0
2,2020-04-22,AC,Cruzeiro do Sul,city,5,0,True,88376.0,1200203.0,5.65764,NaN,Cruzeiro do Sul,-7.62762,-72.6756,0.0,12.0
3,2020-04-22,AC,Plácido de Castro,city,21,1,True,19761.0,1200385.0,106.26993,0.0476,Plácido de Castro,-10.28060,-67.1371,0.0,12.0
4,2020-04-22,AC,Porto Acre,city,1,0,True,18504.0,1200807.0,5.40424,NaN,Porto Acre,-9.58138,-67.5478,0.0,12.0


In [23]:
geo_last = cities.loc [cities.is_last==True,['city','latitude','longitude','state','confirmed','deaths']]

In [24]:
geo_last.head()

,city,latitude,longitude,state,confirmed,deaths
0,Acrelândia,-9.82581,-66.8972,AC,12,0
1,Bujari,-9.81528,-67.9550,AC,1,0
2,Cruzeiro do Sul,-7.62762,-72.6756,AC,5,0
3,Plácido de Castro,-10.28060,-67.1371,AC,21,1
4,Porto Acre,-9.58138,-67.5478,AC,1,0


In [25]:
len(geo_last)

1567

In [26]:
geo_last.confirmed.sum()

45932

In [28]:
len(geo_last.state.unique())

27

In [30]:
coordenadas = geo_last[['latitude','longitude','confirmed']]

In [31]:
coordenadas = coordenadas.dropna()

In [32]:
baseMap = folium.Map(
    width="100%",
    height="100%",
    location=[-15.788497, -47.879873],
    zoom_start= 4
)

In [33]:
baseMap

In [34]:
baseMap = baseMap.add_child(plugins.HeatMap(coordenadas))

In [35]:
baseMap

In [36]:
geo_last = geo_last.dropna()
geo_last

,city,latitude,longitude,state,confirmed,deaths
0,Acrelândia,-9.82581,-66.8972,AC,12,0
1,Bujari,-9.81528,-67.9550,AC,1,0
2,Cruzeiro do Sul,-7.62762,-72.6756,AC,5,0
3,Plácido de Castro,-10.28060,-67.1371,AC,21,1
4,Porto Acre,-9.58138,-67.5478,AC,1,0
...,...,...,...,...,...,...
3229,Viçosa do Ceará,-3.56670,-41.0916,CE,1,0
4680,Jucás,-6.51523,-39.5187,CE,0,0
5385,Silva Jardim,-22.65740,-42.3961,RJ,1,0
6096,Forquilha,-3.79945,-40.2634,CE,1,0


In [41]:
for i in range(0, len(geo_last)):
    folium.Circle(
        location=[geo_last.iloc[i]['latitude'], geo_last.iloc[i]['longitude']],
        color = '#00ff69',
        fill = '#00A1B3',
        tooltip ='<li><bold>CIDADE: '+str(geo_last.iloc[i]['city'])+ "</bold></li>"+
                    '<li><bold>ESTADO: '+str(geo_last.iloc[i]['state'])+ "</bold></li>"+
                    '<li><bold>CASOS: '+str(geo_last.iloc[i]['confirmed'])+ "</bold></li>"+
                    '<li><bold>MORTES: '+str(geo_last.iloc[i]['deaths']) +"</bold></li>",
        radius = (geo_last.iloc[i]['confirmed']**1.1)
    ).add_to(baseMap)

In [42]:
baseMap


In [43]:
baseMap.save('heatmap-cidades.html')